In [7]:
import pandas as pd


# In[83]:
df = pd.read_csv('Data_Train.csv')
df.head(10)
print(df.shape)


(78458, 11)


In [8]:

#Replace K, M, B
def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000
        return 1000000.0
    if 'B' in x:
        return float(x.replace('B', '')) * 1000000000
    if ',' in x:
        return float(x.replace(',', '')) 
    return float(x)

df.Likes = df.Likes.apply(value_to_float)
df.Popularity = df.Popularity.apply(value_to_float)
df.to_csv('Data_Train_Transformed_1.csv')
df = df[df['Views'] > df["Likes"]]
# print(df.shape)
# print(df.head())

In [9]:
df.isnull().sum()

Unique_ID     0
Name          0
Genre         0
Country       0
Song_Name     1
Timestamp     0
Views         0
Comments      0
Likes         0
Popularity    0
Followers     0
dtype: int64

In [10]:
# Distribution of features by Genre
df_mean = df.groupby('Genre').mean()
#df_mean

In [11]:
#Correlation matrix
print(df.head())
df.drop(['Unique_ID','Country','Timestamp','Song_Name','Comments','Followers'],inplace = True, axis = 1)
df.head()

   Unique_ID              Name     Genre Country  \
0     413890         Hardstyle  danceedm      AU   
1     249453        Dj Aladdin  danceedm      AU   
2     681116  Maxximize On Air  danceedm      AU   
3     387253       GR6 EXPLODE    rbsoul      AU   
4    1428029          Tritonal  danceedm      AU   

                                           Song_Name  \
0  N-Vitral presents BOMBSQUAD - Poison Spitter (...   
1          Dj Aladdin - Old School Hip Hop Quick Mix   
2  Maxximize On Air - Mixed by Blasterjaxx - Epis...   
3                MC Yago - Tenho Compromisso (DJ R7)   
4                         Escape (feat. Steph Jones)   

                    Timestamp   Views  Comments   Likes  Popularity  Followers  
0  2018-03-30 15:24:45.000000   14017         4   499.0        97.0     119563  
1  2016-06-20 05:58:52.000000    1918        17    49.0        17.0       2141  
2  2015-05-08 17:45:59.000000    9668        11   312.0        91.0      22248  
3  2017-06-08 23:50:03.000

,Name,Genre,Views,Likes,Popularity
0,Hardstyle,danceedm,14017,499.0,97.0
1,Dj Aladdin,danceedm,1918,49.0,17.0
2,Maxximize On Air,danceedm,9668,312.0,91.0
3,GR6 EXPLODE,rbsoul,113036,2400.0,76.0
4,Tritonal,danceedm,110024,3031.0,699.0


In [12]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
# Converting Genre and Name to labelencoding
labelencoder = LabelEncoder()
df['Genre'] = labelencoder.fit_transform(df['Genre'])
#df['Name'] = labelencoder.fit_transform(df['Name'])
df.head()

,Name,Genre,Views,Likes,Popularity
0,Hardstyle,5,14017,499.0,97.0
1,Dj Aladdin,5,1918,49.0,17.0
2,Maxximize On Air,5,9668,312.0,91.0
3,GR6 EXPLODE,17,113036,2400.0,76.0
4,Tritonal,5,110024,3031.0,699.0


In [364]:
#Hash the Name
from sklearn.feature_extraction import FeatureHasher
fh = FeatureHasher(n_features=10, input_type='string')
hashed_name = fh.fit_transform(df['Name'])
hashed_name = pd.DataFrame(hashed_name.toarray())
hashed_name.head()
df = df.join(hashed_name,rsuffix='N')
df.drop(['Name'],inplace = True, axis = 1)
print(df.shape)
df.head()

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
dtype: int64
(77170, 14)


,Genre,Views,Likes,Popularity,0,1,2,3,4,5,6,7,8,9
0,5,14017,499.0,97.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
1,5,1918,49.0,17.0,1.0,-2.0,3.0,0.0,-1.0,0.0,1.0,0.0,0.0,2.0
2,5,9668,312.0,91.0,2.0,-3.0,4.0,2.0,-1.0,0.0,1.0,0.0,0.0,1.0
3,17,113036,2400.0,76.0,1.0,-2.0,3.0,0.0,0.0,0.0,1.0,2.0,0.0,2.0
4,5,110024,3031.0,699.0,1.0,-2.0,1.0,1.0,-1.0,1.0,0.0,0.0,0.0,-1.0


In [357]:
df.isnull().sum()

Genre            0
Views            0
Likes            0
Popularity       0
0             1269
1             1269
2             1269
3             1269
4             1269
5             1269
6             1269
7             1269
8             1269
9             1269
dtype: int64

In [321]:
# #Hash the Genre
# from sklearn.feature_extraction import FeatureHasher
# fh_genre = FeatureHasher(n_features=4, input_type='string')
# hashed_genre = fh_genre.fit_transform(df['Genre'])
# hashed_genre = pd.DataFrame(hashed_genre.toarray())
# hashed_genre.head()
# df = df.join(hashed_genre, hashed_name)
# df.drop(['Genre'],inplace = True, axis = 1)
# print(df.shape)
# df.head()

In [325]:
#Standard scaling
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
sc_x = StandardScaler()
ct = ColumnTransformer([
         ('somename', StandardScaler(), ['Genre','Likes', 'Popularity',[0],[1],[2],[3],[4],[5],[6],[7],[8],[9]])
       ], remainder='passthrough')
df = ct.fit_transform(df)
df = pd.DataFrame(df,columns=['Genre','Likes','Popularity','Views','N1','N2','N3','N4','N5','N6','N7','N8','N9','N10'])
print(df.head())

ValueError: No valid specification of the columns. Only a scalar, list or slice of all integers or all strings, or boolean mask is allowed

In [ ]:
from scipy import stats
import numpy as np
df['like_z_score'] = np.abs(stats.zscore(df['Likes']))
df= df[df['like_z_score']<3]
df['pop_z_score'] = np.abs(stats.zscore(df['Popularity']))
df= df[df['pop_z_score']<3]
# df['comment_z_score'] = np.abs(stats.zscore(df['Comments']))
# df= df[df['comment_z_score']<3]
df['views_z_score'] = np.abs(stats.zscore(df['Views']))
df= df[df['views_z_score']<3]
df.drop(['like_z_score','pop_z_score','views_z_score'],inplace = True, axis = 1)
df.shape

In [ ]:
#Histogram Plotting each variable
#Plotting Likes or comments or popularity
import matplotlib.pyplot as plt
import seaborn as sns
plt.hist(df['Likes'])
plt.ylabel('No of times')
plt.show()

# #Plotting Comments
# plt.hist(df['Comments'])
# plt.ylabel('No of times')
# plt.show()

#Plotting Popularity
plt.hist(df['Popularity'])
plt.ylabel('No of times')
plt.show()

#Plotting Views
plt.hist(df['Views'])
plt.ylabel('No of times')
plt.show()

In [ ]:
# Scatter Plot
fig, ax = plt.subplots(figsize=(16,8))
ax.scatter(df['Unique_ID'], df['Popularity'])
ax.set_xlabel('Unique Id')
ax.set_ylabel('Popularity')

fig, ax = plt.subplots(figsize=(16,8))
ax.scatter(df['Unique_ID'], df['Views'])
ax.set_xlabel('Unique Id')
ax.set_ylabel('Views')

fig, ax = plt.subplots(figsize=(16,8))
ax.scatter(df['Unique_ID'], df['Likes'])
ax.set_xlabel('Unique Id')
ax.set_ylabel('Likes')

In [ ]:
df.isnull().sum()


In [ ]:
y = df['Views']
df.drop('Views',inplace = True, axis = 1)
y.plot.hist()

In [ ]:
df.describe()
df.plot.hist()

In [ ]:
from sklearn.model_selection import train_test_split
#print(df_scaled)
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=.2, random_state=205)

In [ ]:
#XGB Boost regression
import xgboost as xgb
from sklearn import metrics
xlf = xgb.XGBRegressor()
xlf.fit(X_train, y_train, eval_metric='rmse', verbose = True, eval_set = [(X_test, y_test)])
# calculate the auc score
y_pred = xlf.predict(X_test)
#RMSE - Test
print("XGBoost - Linear regression: ", np.sqrt(metrics.mean_squared_error(y_test,y_pred)))


In [ ]:
#Implementing Linear Regression
from sklearn.linear_model import LinearRegression
from sklearn import metrics

linear = LinearRegression()
linear.fit(X_train, y_train)
y_pred = linear.predict(X_test)
print("RMSE - Linear regression: ", np.sqrt(metrics.mean_squared_error(y_test,y_pred)))
